# **Juan Daniel Ramírez Mojica - LP02 Tarea 16**

In [1]:
class Memory:
    def __init__(self, size=1024):
        self.data = [0] * size  # Inicializa la memoria con ceros

    def read(self, address):
        if 0 <= address < len(self.data):
            return self.data[address]
        else:
            raise IndexError(f"Invalid memory address: {address}")

    def write(self, address, value):
        if 0 <= address < len(self.data):
            self.data[address] = value
        else:
            raise IndexError(f"Invalid memory address: {address}")


class ALU:
    def execute(self, operation, operand1, operand2):
        if operation == "ADD":
            return operand1 + operand2
        elif operation == "SUB":
            return operand1 - operand2
        elif operation == "MUL":
            return operand1 * operand2
        elif operation == "DIV":
            if operand2 == 0:
                raise ZeroDivisionError("Division by zero is undefined")
            return operand1 // operand2
        else:
            raise ValueError(f"Unsupported operation: {operation}")


class CPU:
    def __init__(self, memory):
        self.registers = {"A": 0, "B": 0, "C": 0, "D": 0}
        self.pc = 0
        self.memory = memory
        self.alu = ALU()
        self.instructions = {
            0b0001: self.load,  # LOAD
            0b0010: self.store,  # STORE
            0b0011: self.add,    # ADD
            0b0100: self.sub,    # SUB
            0b0101: self.mul,    # MUL
            0b0110: self.div,    # DIV
            0b0111: self.jump,   # JUMP
            0b1000: self.halt,   # HALT
        }

    def fetch(self):
        instruction = self.memory.read(self.pc)
        self.pc += 1
        return instruction

    def decode(self, instruction):
        # Extraemos el opcode (4 bits), reg1 (4 bits), reg2 (4 bits) y addr_or_val (20 bits)
        op_code = (instruction >> 28) & 0xF
        reg1 = (instruction >> 24) & 0xF
        reg2 = (instruction >> 20) & 0xF
        addr_or_val = instruction & 0xFFFFF
        return op_code, reg1, reg2, addr_or_val

    def execute(self, op_code, reg1, reg2, addr_or_val):
        if op_code in self.instructions:
            self.instructions[op_code](reg1, reg2, addr_or_val)
        else:
            raise ValueError(f"Unknown instruction: {op_code}")

    def get_register_name(self, reg_index):
        reg_names = ["A", "B", "C", "D"]
        if 0 <= reg_index < len(reg_names):
            return reg_names[reg_index]
        else:
            raise KeyError(f"Invalid register index: {reg_index}")

    def load(self, reg, _, addr):
        # La instrucción LOAD debe cargar el valor de la memoria en el registro
        reg_name = self.get_register_name(reg)
        value = self.memory.read(addr)  # Leemos el valor de la memoria en la dirección `addr`
        print(f"Loading {value} into {reg_name} from address {addr}")
        self.registers[reg_name] = value  # Cargamos el valor en el registro

    def store(self, reg, _, addr):
        # La instrucción STORE debe almacenar el valor del registro en la memoria
        reg_name = self.get_register_name(reg)
        value = self.registers[reg_name]
        print(f"Storing {value} from {reg_name} into address {addr}")
        self.memory.write(addr, value)  # Escribimos el valor del registro en la memoria

    def add(self, reg1, reg2, _):
        reg1_name = self.get_register_name(reg1)
        reg2_name = self.get_register_name(reg2)
        print(f"Adding {self.registers[reg1_name]} + {self.registers[reg2_name]}")
        self.registers[reg1_name] = self.alu.execute("ADD", self.registers[reg1_name], self.registers[reg2_name])

    def sub(self, reg1, reg2, _):
        reg1_name = self.get_register_name(reg1)
        reg2_name = self.get_register_name(reg2)
        print(f"Subtracting {self.registers[reg1_name]} - {self.registers[reg2_name]}")
        self.registers[reg1_name] = self.alu.execute("SUB", self.registers[reg1_name], self.registers[reg2_name])

    def mul(self, reg1, reg2, _):
        reg1_name = self.get_register_name(reg1)
        reg2_name = self.get_register_name(reg2)
        print(f"Multiplying {self.registers[reg1_name]} * {self.registers[reg2_name]}")
        self.registers[reg1_name] = self.alu.execute("MUL", self.registers[reg1_name], self.registers[reg2_name])

    def div(self, reg1, reg2, _):
        reg1_name = self.get_register_name(reg1)
        reg2_name = self.get_register_name(reg2)
        print(f"Dividing {self.registers[reg1_name]} / {self.registers[reg2_name]}")
        self.registers[reg1_name] = self.alu.execute("DIV", self.registers[reg1_name], self.registers[reg2_name])

    def jump(self, _, __, addr):
        print(f"Jumping to address {addr}")
        self.pc = addr

    def halt(self, _, __, ___):
        print(f"Program halted. Final value in register A: {self.registers['A']}")

    def run(self, program):
        self.pc = 0
        for addr, instruction in enumerate(program):
            self.memory.write(addr, instruction)
        running = True
        while running:
            instruction = self.fetch()
            op_code, reg1, reg2, addr_or_val = self.decode(instruction)
            print(f"Fetching instruction at PC={self.pc-1}: {instruction}")
            print(f"Decoded: op_code={op_code}, reg1={reg1}, reg2={reg2}, addr_or_val={addr_or_val}")
            if op_code == 0b1000:  # HALT
                running = False
            self.execute(op_code, reg1, reg2, addr_or_val)
        print("Execution finished.")


memory = Memory()
cpu = CPU(memory)


# Cada instrucción debe ser un valor de 32 bits.
# Codificación: op_code (4 bits) | reg1 (4 bits) | reg2 (4 bits) | addr_or_val (20 bits)

program = [
    0b00010000_00000000_00000000_00000001,  # LOAD A, 1 (Cargar el valor 1 en A)
    0b00010001_00000000_00000000_00000010,  # LOAD B, 2 (Cargar el valor 2 en B)
    0b00110000_00000001_00000000_00000000,  # ADD A, B (Sumar A y B)
    0b10000000_00000000_00000000_00000000,  # HALT (Detener el programa)
]

# Correr
cpu.run(program)

Fetching instruction at PC=0: 268435457
Decoded: op_code=1, reg1=0, reg2=0, addr_or_val=1
Loading 285212674 into A from address 1
Fetching instruction at PC=1: 285212674
Decoded: op_code=1, reg1=1, reg2=0, addr_or_val=2
Loading 805371904 into B from address 2
Fetching instruction at PC=2: 805371904
Decoded: op_code=3, reg1=0, reg2=0, addr_or_val=65536
Adding 285212674 + 285212674
Fetching instruction at PC=3: 2147483648
Decoded: op_code=8, reg1=0, reg2=0, addr_or_val=0
Program halted. Final value in register A: 570425348
Execution finished.


# **Escenario 1:**

In [4]:
program = [
    0b00010000_00000000_00000000_00000001,  # LOAD A, 1
    0b00010001_00000000_00000000_00000010,  # LOAD B, 2
    0b00110000_00000001_00000000_00000000,  # ADD A, B
    0b10000000_00000000_00000000_00000000,  # HALT
]


cpu.run(program)

Fetching instruction at PC=0: 268435457
Decoded: op_code=1, reg1=0, reg2=0, addr_or_val=1
Loading 285212674 into A from address 1
Fetching instruction at PC=1: 285212674
Decoded: op_code=1, reg1=1, reg2=0, addr_or_val=2
Loading 805371904 into B from address 2
Fetching instruction at PC=2: 805371904
Decoded: op_code=3, reg1=0, reg2=0, addr_or_val=65536
Adding 285212674 + 285212674
Fetching instruction at PC=3: 2147483648
Decoded: op_code=8, reg1=0, reg2=0, addr_or_val=0
Program halted. Final value in register A: 570425348
Execution finished.


# **Escenario 2:**

In [5]:
# Programa para el Segundo Escenario: Acceso a Memoria
program = [
    0b00010000_00000000_00000000_00000001,  # LOAD A, [1] (Cargar en A el valor en la dirección de memoria 1)
    0b00010001_00000000_00000000_00000010,  # LOAD B, [2] (Cargar en B el valor en la dirección de memoria 2)
    0b00110000_00000001_00000000_00000000,  # ADD A, B (Sumar A y B)
    0b10000000_00000000_00000000_00000000,  # HALT (Detener el programa)
]

# Valores en la memoria
memory.write(1, 5)  # Dirección 1 contiene el valor 5
memory.write(2, 10) # Dirección 2 contiene el valor 10

# Ejecutar el programa
cpu.run(program)


Fetching instruction at PC=0: 268435457
Decoded: op_code=1, reg1=0, reg2=0, addr_or_val=1
Loading 285212674 into A from address 1
Fetching instruction at PC=1: 285212674
Decoded: op_code=1, reg1=1, reg2=0, addr_or_val=2
Loading 805371904 into B from address 2
Fetching instruction at PC=2: 805371904
Decoded: op_code=3, reg1=0, reg2=0, addr_or_val=65536
Adding 285212674 + 285212674
Fetching instruction at PC=3: 2147483648
Decoded: op_code=8, reg1=0, reg2=0, addr_or_val=0
Program halted. Final value in register A: 570425348
Execution finished.


# **Tercer Escenario: Algoritmo de Euclides**

In [3]:


program = [
    0b00010000_00000000_00000000_00000001,  # LOAD A, 1 (Cargar el primer número en A)
    0b00010001_00000000_00000000_00000010,  # LOAD B, 2 (Cargar el segundo número en B)
    0b01000000_00000001_00000000_00000000,  # SUB A, B (A = A - B)
    0b01110000_00000000_00000000_00000110,  # JUMP, 6 si A >= 0
    0b01000001_00000000_00000000_00000000,  # SUB B, A (B = B - A)
    0b01110000_00000000_00000000_00000011,  # JUMP, 3 (volver al inicio del ciclo)
    0b10000000_00000000_00000000_00000000,  # HALT (A o B contiene el MCD)
]

# Ejecutar el programa
memory.write(1, 48)  # Primer número
memory.write(2, 18)  # Segundo número
cpu.run(program)



Fetching instruction at PC=0: 268435457
Decoded: op_code=1, reg1=0, reg2=0, addr_or_val=1
Loading 285212674 into A from address 1
Fetching instruction at PC=1: 285212674
Decoded: op_code=1, reg1=1, reg2=0, addr_or_val=2
Loading 1073807360 into B from address 2
Fetching instruction at PC=2: 1073807360
Decoded: op_code=4, reg1=0, reg2=0, addr_or_val=65536
Subtracting 285212674 - 285212674
Fetching instruction at PC=3: 1879048198
Decoded: op_code=7, reg1=0, reg2=0, addr_or_val=6
Jumping to address 6
Fetching instruction at PC=6: 2147483648
Decoded: op_code=8, reg1=0, reg2=0, addr_or_val=0
Program halted. Final value in register A: 0
Execution finished.


# **Cuarto: Factorial**

Se queda en un bulce infinito:

In [3]:
program = [
    0b00010000_00000000_00000000_00000001,  # LOAD A, 1 (Cargar el número en A)
    0b00010001_00000000_00000000_00000001,  # LOAD B, 1 (Inicializar B como 1 para multiplicar)
    0b01000001_00000000_00000000_00000000,  # MUL A, B (A = A * B)
    0b01000000_00000001_00000000_00000001,  # SUB A, B (A = A - 1)
    0b01110000_00000000_00000000_00000011,  # JUMP, 3 si A > 1 (seguir multiplicando)
    0b10000000_00000000_00000000_00000000,  # HALT (A contiene el factorial)
]


cpu.run(program)


Se han truncado las últimas 5000 líneas del flujo de salida.
Fetching instruction at PC=3: 1073807361
Decoded: op_code=4, reg1=0, reg2=0, addr_or_val=65537
Subtracting 0 - 0
Fetching instruction at PC=4: 1879048195
Decoded: op_code=7, reg1=0, reg2=0, addr_or_val=3
Jumping to address 3
Fetching instruction at PC=3: 1073807361
Decoded: op_code=4, reg1=0, reg2=0, addr_or_val=65537
Subtracting 0 - 0
Fetching instruction at PC=4: 1879048195
Decoded: op_code=7, reg1=0, reg2=0, addr_or_val=3
Jumping to address 3
Fetching instruction at PC=3: 1073807361
Decoded: op_code=4, reg1=0, reg2=0, addr_or_val=65537
Subtracting 0 - 0
Fetching instruction at PC=4: 1879048195
Decoded: op_code=7, reg1=0, reg2=0, addr_or_val=3
Jumping to address 3
Fetching instruction at PC=3: 1073807361
Decoded: op_code=4, reg1=0, reg2=0, addr_or_val=65537
Subtracting 0 - 0
Fetching instruction at PC=4: 1879048195
Decoded: op_code=7, reg1=0, reg2=0, addr_or_val=3
Jumping to address 3
Fetching instruction at PC=3: 107380736

KeyboardInterrupt: 